In [1]:
#will work on the temperature arrays in here
#once succeeded, will put back into the main file

import context
from Integrator_Functions import Integrator
from collections import namedtuple
import numpy as np
import matplotlib.pyplot as plt

class Integ71(Integrator):
    def set_yinit(self):
        
        # TODO: We will need to set up the initial conditions here in:
        constants = namedtuple('constants', ['solar', 'albedo'])
        self.initvars = constants(np.array([1,2,3,4]), np.array([4,3,2,1]))

        ##############################################################################
        ## COMPLETED: Change the arrays below to include the initial temperature array:

        # Information from the table
        table_Ti = np.array([26.4, 26.1, 22.9, 16.2, 8.8, 2.2, -5.1, -12.3, -16.9])
        table_lats = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])

        # Create an array for the latitudes from 0° to 90°N with 91 values
        latitudes = np.arange(0, 91, 1) 

        # Linearly interpolate temperature values
        interpolated_T = np.interp(latitudes, table_lats, table_Ti)
     
        # Set initial temperature array
        initvars = namedtuple('initvars', ['temperature'])
        self.yinit = initvars(temperature=interpolated_T)
        ##############################################################################
        
        # TODO: Make sure that we set this to the initial temperature array length 90
        self.nvars = 4
        return None

    def __init__(self, coeffFileName):
        super().__init__(coeffFileName)
        self.set_yinit()

    def derivs5(self, y, t):
        """arrs[0]=solar insolation
           arrs[1]=albedo

           Constant growty rates for white
           and black daisies beta_w and beta_b

           returns dy/dt
        """
        ## TODO: Eventually add the derivative here. Also need to figure out where to store constants,
        ## likely in constants tuple for initvars or another tuple
        arrs = self.initvars
        f = arrs.solar
        return f
    

******************************
context imported. Front of path:
/Users/janesharai/repos
back of path: /Users/janesharai/miniconda3/envs/numeric_2024/lib/python3.12/site-packages
******************************

through /Users/janesharai/repos/numeric_2024/ATSC409/context.py


In [2]:
##Outside the stuff 

#Create an initial temperature array, 90 values corresponding for lattidues from 0° (Equator) to 90°N (North Pole)
#Considering the Northern Hemisphere

#Information from the table
table_Ti = np.array([26.4, 26.1, 22.9, 16.2, 8.8, 2.2, -5.1, -12.3, -16.9])
table_lats = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])

#linearly interpolate between for temperature values

latitudes = np.arange(0, 91, 1) ##QUESTION: Shouldn't our arrays have 91 values in them, not 90 (as we are also considering 0° latitude?)
interpolated_T = np.interp(latitudes, table_lats, table_Ti)

np.set_printoptions(suppress=True) #to see the values without scientific notation - can comment out later
print(interpolated_T)

[ 26.4   26.4   26.4   26.4   26.4   26.4   26.4   26.4   26.4   26.4
  26.4   26.37  26.34  26.31  26.28  26.25  26.22  26.19  26.16  26.13
  26.1   25.78  25.46  25.14  24.82  24.5   24.18  23.86  23.54  23.22
  22.9   22.23  21.56  20.89  20.22  19.55  18.88  18.21  17.54  16.87
  16.2   15.46  14.72  13.98  13.24  12.5   11.76  11.02  10.28   9.54
   8.8    8.14   7.48   6.82   6.16   5.5    4.84   4.18   3.52   2.86
   2.2    1.47   0.74   0.01  -0.72  -1.45  -2.18  -2.91  -3.64  -4.37
  -5.1   -5.82  -6.54  -7.26  -7.98  -8.7   -9.42 -10.14 -10.86 -11.58
 -12.3  -12.76 -13.22 -13.68 -14.14 -14.6  -15.06 -15.52 -15.98 -16.44
 -16.9 ]


In [3]:
def create_interp_solar(lat):
    """
    Input: takes in the lat array.
    Output: spits out the interpolated solar insolation array
    """
    deg2rad = np.pi/180
    s_i = 1366*np.cos(lat*deg2rad) #solar constant is dependant per latitude

    table_solar_fractions = np.array([1.219, 1.189, 1.120, 1.021, 0.892, 0.770, 0.624, 0.531, 0.500])
    table_lats = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
    interp_solar_fractions = np.interp(lat, table_lats, table_solar_fractions)

    interp_solar_insolation = (s_i/4)*interp_solar_fractions

    return interp_solar_insolation

In [6]:
lat = np.arange(0, 91, 1) 

deg2rad = np.pi/180
s_i = 1366*np.cos(lat*deg2rad) #solar constant is dependant per latitude

table_solar_fractions = np.array([1.219, 1.189, 1.120, 1.021, 0.892, 0.770, 0.624, 0.531, 0.500])
table_lats = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])
interp_solar_fractions = np.interp(lat, table_lats, table_solar_fractions)

interp_solar_insolation = (s_i/4)*interp_solar_fractions



[416.2885     416.22509725 416.03490829 415.71799108 415.27444214
 414.70439658 414.00802804 413.18554865 412.23720893 411.16329777
 409.96414229 407.63443071 405.18737305 402.62432632 399.94668056
 397.15585829 394.25331386 391.24053285 388.11903138 384.89035553
 381.55608067 376.87442167 372.1074381  367.25791763 362.32866367
 357.32249416 352.24224023 347.09074504 341.87086244 336.58545573
 331.23739644 324.9513951  318.62718901 312.26846066 305.87888404
 299.46212295 293.02182938 286.56164183 280.08518379 273.59606202
 267.09786505 259.82096155 252.5657949  245.33658509 238.13751284
 230.97271776 223.8462965  216.76230087 209.72473608 202.73755894
 195.80467609 189.08038134 182.41150648 175.80166293 169.2544092
 162.77324935 156.36163151 150.02294629 143.76052539 137.57764012
 131.4775     125.06590141 118.76841826 112.58841816 106.5291861
 100.59392306  94.78574461  89.10767956  83.56266866  78.15356334
  72.88312446  68.34328326  63.88744039  59.51756704  55.23557613
  51.0433215